## Multi-task UnitedNet on perturbation prediction with the PBMC dataset

Motivation relies on the potential of scButterfly for perturbation response on the PBMC dataset stimulated by IFN-beta. scButterfly and UnitedNet share similar architectures. The seconds one leverages multi-task learning. Thus, we will attempt for a simple dataset with one perturbation type to explore the usage of UnitedNet with the tasks:

- cross modal prediction on perturbation (perturbation response prediction task)
- classification task for cell type annotation

We would like to investigate if multi-task can be beneficial for perturbation modeling. Thus we will compare the multi-task model with the single-task models of the classification and the cross modal prediction task.

In [29]:
# prepare dataset
# dataset from https://github.com/theislab/scgen-reproducibility, already preprocessed

import scanpy as sc

train_data = "../data/pbmc/train_pbmc.h5ad"
valid_data = "../data/pbmc/valid_pbmc.h5ad"

# pbmc data from scGen is already split into train and valid, but we will concatenate them to do a custom split
sc_data_train = sc.read_h5ad(train_data)
sc_data_valid = sc.read_h5ad(valid_data)
pbmc = sc_data_train.concatenate(sc_data_valid)

In [30]:
pbmc

AnnData object with n_obs × n_vars = 18868 × 6998
    obs: 'condition', 'n_counts', 'n_genes', 'mt_frac', 'cell_type', 'batch'
    var: 'gene_symbol', 'n_cells'
    obsm: 'X_pca', 'X_tsne', 'X_umap'

In [31]:
pbmc.obs.groupby(['cell_type', 'condition']).size()

cell_type    condition 
CD4T         control       2715
             stimulated    3483
CD14+Mono    control       2184
             stimulated     698
B            control        928
             stimulated    1105
CD8T         control        643
             stimulated     594
NK           control        571
             stimulated     733
FCGR3A+Mono  control       1232
             stimulated    2790
Dendritic    control        670
             stimulated     522
dtype: int64

In [32]:
pbmc.obs.groupby(['condition']).size()

condition
control       8943
stimulated    9925
dtype: int64

In [33]:
# simple inspection

# sc.pp.pca(pbmc)
# sc.pp.neighbors(pbmc)
# sc.tl.tsne(pbmc)
# sc.pl.tsne(pbmc, color=['condition', 'cell_type'], legend_loc='on data', legend_fontsize='small')

### Create pairs of control and stimulated data using scButterfly method

scButterfly showed the potential of using a dual VAR scheme for perturbation response prediction. They split the cells based on the cell type for control and stimulated, and then per cell type group, they create pairs of control and stimulated gene expression profile using optimal transport.

It should be noted that during perturbation, we can't measure the exact same cell that was perturbed. The current experimental methods, don't allow this type of information. Thus, deep learning architectures that are relying on pairs, such as scButterfly and UnitedNet (combining modalities), need to find a strategy to explore if creating pseudo pairs is beneficial.

scButterfly showed this exact potential.

In [34]:
from scButterfly.split_datasets import unpaired_split_dataset_perturb


# create pairs of data using scButterfly technique
control_data = pbmc[pbmc.obs.condition == 'control']
stimulate_data = pbmc[pbmc.obs.condition == 'stimulated']

control_data.obs.index = [str(i) for i in range(control_data.X.shape[0])]
stimulate_data.obs.index = [str(i) for i in range(stimulate_data.X.shape[0])]

cell_type_list = list(control_data.obs.cell_type.cat.categories)

In [35]:
id_list, id_list_dict = unpaired_split_dataset_perturb(control_data, stimulate_data)

optimal transport array torch.Size([2715, 3483])
CD4T, control num 2715 stimulate num 3483

optimal transport array torch.Size([2184, 698])
CD14+Mono, control num 2184 stimulate num 698

optimal transport array torch.Size([928, 1105])
B, control num 928 stimulate num 1105

optimal transport array torch.Size([643, 594])
CD8T, control num 643 stimulate num 594

optimal transport array torch.Size([571, 733])
NK, control num 571 stimulate num 733

optimal transport array torch.Size([1232, 2790])
FCGR3A+Mono, control num 1232 stimulate num 2790

optimal transport array torch.Size([670, 522])
Dendritic, control num 670 stimulate num 522


Start CD4T
Batch list ['Dendritic', 'B', 'FCGR3A+Mono', 'CD14+Mono', 'CD4T', 'NK', 'CD8T']
Test batch control ['Dendritic']
Validation batch control ['B', 'FCGR3A+Mono', 'CD14+Mono', 'CD4T', 'NK', 'CD8T']
Train batch control ['B', 'FCGR3A+Mono', 'CD14+Mono', 'CD4T', 'NK', 'CD8T']
Test batch stimulated ['Dendritic']
Validation batch stimulated ['B', 'FCGR3A+

In [36]:
#print(len(id_list[0]))
for idx, cell_type in enumerate(cell_type_list):
    print(cell_type)
    train_id_control, train_id_peturb, validation_id_control, validation_id_peturb, test_id_control, test_id_peturb = id_list[idx]

    control = len(train_id_control) + len(validation_id_control) + len(test_id_control)
    peturb = len(train_id_peturb) + len(validation_id_peturb) + len(test_id_peturb) 
    control_dict = id_list_dict[cell_type]["control"]
    peturb_dict = id_list_dict[cell_type]["stimulated"]
    print("control", len(train_id_control), len(validation_id_control), len(test_id_control), control)
    print("control", len(control_dict["train"]), len(control_dict["validation"]), len(control_dict["test"]))
    print("peturb", len(train_id_peturb), len(validation_id_peturb), len(test_id_peturb), peturb)    
    print("peturb", len(peturb_dict["train"]), len(peturb_dict["validation"]), len(peturb_dict["test"]))
    print()
    
# Some notes due to the lack of code readability from scButterfly:
# - the rows of the id_list correspond to the cell types, but they don't have any meaning.
# - the rows actually correspond to k-fold cross validation. One batch is a cell type that is held out.
# - the cell_type of the row and the one that is held out are irrelevant.
# - for the test, the control and the stimulated can have different size. But for train and validation we need pairs.

CD4T
control 6616 1657 670 8943
control 6616 1657 670
peturb 6616 1657 522 8795
peturb 6616 1657 522

CD14+Mono
control 6410 1605 928 8943
control 6410 1605 928
peturb 6410 1605 1105 9120
peturb 6410 1605 1105

B
control 6167 1544 1232 8943
control 6167 1544 1232
peturb 6167 1544 2790 10501
peturb 6167 1544 2790

CD8T
control 5405 1354 2184 8943
control 5405 1354 2184
peturb 5405 1354 698 7457
peturb 5405 1354 698

NK
control 4980 1248 2715 8943
control 4980 1248 2715
peturb 4980 1248 3483 9711
peturb 4980 1248 3483

FCGR3A+Mono
control 6696 1676 571 8943
control 6696 1676 571
peturb 6696 1676 733 9105
peturb 6696 1676 733

Dendritic
control 6638 1662 643 8943
control 6638 1662 643
peturb 6638 1662 594 8894
peturb 6638 1662 594



In [41]:
# create adatas for pair of control and stimulated
import numpy as np


batch = cell_type_list[0]
id_list_batch = id_list_dict[batch]
control = id_list_batch["control"]
peturb = id_list_batch["stimulated"]
train_id_control = control["train"]
validation_id_control = control["validation"]
test_id_control = control["test"]
train_id_peturb = peturb["train"]
validation_id_peturb = peturb["validation"]
test_id_peturb = peturb["test"]

control_train = pbmc[train_id_control]
control_valid = pbmc[validation_id_control]
control_test = pbmc[test_id_control]

peturb_train = pbmc[train_id_peturb]
peturb_valid = pbmc[validation_id_peturb]
peturb_test = pbmc[test_id_peturb]

all = [control_train, control_valid, control_test, peturb_train, peturb_valid, peturb_test]
for adata in all:
    adata.obs["label"] = list(adata.obs["cell_type"])
    adata.X = adata.X.toarray()
    print("min", np.max(adata.X))
    print("max", np.min(adata.X))
    print()

adatas_train = [control_train, peturb_train]
adatas_valid = [control_valid, peturb_valid]
adatas_test = [control_test, peturb_test]

features_num = adatas_train[0].n_vars
adatas_test


min 7.0346947
max 0.0

min 6.822089
max 0.0

min 6.7637987
max 0.0

min 6.879698
max 0.0

min 7.0346947
max 0.0

min 5.961386
max 0.0



[AnnData object with n_obs × n_vars = 670 × 6998
     obs: 'condition', 'n_counts', 'n_genes', 'mt_frac', 'cell_type', 'batch', 'label'
     var: 'gene_symbol', 'n_cells'
     obsm: 'X_pca', 'X_tsne', 'X_umap',
 AnnData object with n_obs × n_vars = 522 × 6998
     obs: 'condition', 'n_counts', 'n_genes', 'mt_frac', 'cell_type', 'batch', 'label'
     var: 'gene_symbol', 'n_cells'
     obsm: 'X_pca', 'X_tsne', 'X_umap']

### Prepare multi-task model

UnitedNet attempts to solve two tasks, the cell type annotation and the cross modal prediction.

Metrics overview:

Translation
- R2

Classification
- Confusion matrix
- Adjusted Rank Index
- Accuracy
- Normalized Mutual information

Both of the classification and the translation network's parameters are taken into account in the multi-task way orchestrated by UnitedNet.

In [40]:
from unitednet.interface import UnitedNet


pbmc_config = {
    "train_batch_size": 256,
    "finetune_batch_size": 5000,
    "transfer_batch_size": 512,
    "train_epochs": 50,
    "finetune_epochs": 10,
    "transfer_epochs": 20,
    "train_task": "supervised_group_identification", # -> translation, classification
    "finetune_task": None,
    "transfer_task": None,
    "train_loss_weight": None,
    "finetune_loss_weight": None,
    "transfer_loss_weight": None,
    "lr": 0.1,
    "checkpoint": 1,
    "n_head": 1,
    "noise_level":[0,0],
    "fuser_type":"WeightedMean",
    "encoders": [
        {
            "input": features_num,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
        {
            "input": features_num,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
    ],
    "latent_projector": None,
    "decoders": [
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": features_num,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "sigmoid"],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": features_num,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", None],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
    ],
    "discriminators": [
        {
            "input": features_num,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
        {
            "input": features_num,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
    ],
    "projectors": {
        "input": 64,
        "hiddens": [],
        "output": 100,
        "use_biases": [True],
        "dropouts": [0],
        "activations": ["relu"],
        "use_batch_norms": [False],
        "use_layer_norms": [True],
    },
    "clusters": {
        "input": 100,
        "hiddens": [],
        "output": len(cell_type_list),
        "use_biases": [False],
        "dropouts": [0],
        "activations": [None],
        "use_batch_norms": [False],
        "use_layer_norms": [False],
    },
}

device="cuda:0"
test_batch= batch
root_save_path = f"saved_results/thesis/pbmc/multi_task"


model = UnitedNet("multitask", f"{root_save_path}/{test_batch}", device=device, technique=pbmc_config)
model.train(adatas_train=adatas_train)
#model.finetune(adatas_train)
#model.transfer(adatas_train, adatas_transfer = adatas_test, verbose=True)
#print(model.evaluate(adatas_test))
# evaluation should be different for peturbation

training


  0%|          | 0/50 [00:00<?, ?it/s]

model saved at saved_results/thesis/pbmc/multi_task/CD4T/train_epoch_1.pt 



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05406102 -0.05405913]
                                [-1.99397302 -1.99397302]]
Confusion Matrix               [[ 745    0    0    0    0    0    0]
                                [   0  435    0    0    0   18    0]
                                [   0    0 2326   23    0    0    0]
                                [   0    0    0  390    0    0    0]
                                [   3    0    0    0  332    4    0]
                                [   2   32    1    0    4 1820    0]
                                [   0    0    0    2    0    1  478]]
Accuracy                       0.9863966142684402
Adjusted Rand Index            0.973166750722691
Normalized Mutual Information  0.9592932191659395


  2%|▏         | 1/50 [00:07<05:52,  7.19s/it]

model saved at saved_results/thesis/pbmc/multi_task/CD4T/train_epoch_2.pt 



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05406102 -0.05406102]
                                [-0.13593295 -0.13593295]]
Confusion Matrix               [[ 745    0    0    0    0    0    0]
                                [   0  453    0    0    0    0    0]
                                [   1    0 2330   16    2    0    0]
                                [   0    0    0  388    0    0    2]
                                [   0    0    0    0  335    4    0]
                                [   0    8    0    0    0 1851    0]
                                [   0    0    0    0    0    0  481]]
Accuracy                       0.9950120918984281
Adjusted Rand Index            0.9896526460167843
Normalized Mutual Information  0.9829681328421657


...

 96%|█████████▌| 48/50 [05:39<00:13,  6.90s/it]

model saved at saved_results/thesis/pbmc/multi_task/CD4T/train_epoch_49.pt 



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05786815 -0.06501304]
                                [ 0.00497342  0.00497342]]
Confusion Matrix               [[ 745    0    0    0    0    0    0]
                                [   0  453    0    0    0    0    0]
                                [   0    0 2349    0    0    0    0]
                                [   0    0    0  390    0    0    0]
                                [   0    0    0    0  339    0    0]
                                [   0    0    0    0    0 1859    0]
                                [   0    0    0    0    0    0  481]]
Accuracy                       1.0
Adjusted Rand Index            1.0
Normalized Mutual Information  1.0


 98%|█████████▊| 49/50 [05:46<00:06,  6.98s/it]

model saved at saved_results/thesis/pbmc/multi_task/CD4T/train_epoch_50.pt 



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05786815 -0.07130054]
                                [ 0.00449098  0.00449098]]
Confusion Matrix               [[ 745    0    0    0    0    0    0]
                                [   0  453    0    0    0    0    0]
                                [   0    0 2349    0    0    0    0]
                                [   0    0    0  390    0    0    0]
                                [   0    0    0    0  339    0    0]
                                [   0    0    0    0    0 1859    0]
                                [   0    0    0    0    0    0  481]]
Accuracy                       1.0
Adjusted Rand Index            1.0
Normalized Mutual Information  1.0


100%|██████████| 50/50 [05:53<00:00,  7.07s/it]


## Single-task translation via UnitedNet 

The classification network's parameters are ignored.

It should be noted that all the metrics used in the multi-task way are still monitored for the single task as well. But we are only interested for the translation, so the metrics to pay attention to is the R2.

In [16]:
from unitednet.interface import UnitedNet


pbmc_config_translation = {
    "train_batch_size": 512,
    "finetune_batch_size": 5000,
    "transfer_batch_size": 512,
    "train_epochs": 10,
    "finetune_epochs": 10,
    "transfer_epochs": 20,
    "train_task": "cross_model_prediction", # translation
    "finetune_task": None,
    "transfer_task": None,
    "train_loss_weight": None,
    "finetune_loss_weight": None,
    "transfer_loss_weight": None,
    "lr": 0.1,
    "checkpoint": 1,
    "n_head": 1,
    "noise_level":[0,0],
    "fuser_type":"WeightedMean",
    "encoders": [
        {
            "input": features_num,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
        {
            "input": features_num,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
    ],
    "latent_projector": None,
    "decoders": [
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": features_num,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "sigmoid"],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": features_num,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", None],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
    ],
    "discriminators": [
        {
            "input": features_num,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
        {
            "input": features_num,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
    ],
    "projectors": {
        "input": 64,
        "hiddens": [],
        "output": 100,
        "use_biases": [True],
        "dropouts": [0],
        "activations": ["relu"],
        "use_batch_norms": [False],
        "use_layer_norms": [True],
    },
    "clusters": {
        "input": 100,
        "hiddens": [],
        "output": len(cell_type_list),
        "use_biases": [False],
        "dropouts": [0],
        "activations": [None],
        "use_batch_norms": [False],
        "use_layer_norms": [False],
    },
}

device="cuda:0"
test_batch= batch
root_save_path = f"saved_results/thesis/pbmc/translation"

model_translation = UnitedNet(f"{root_save_path}/{test_batch}", device=device, technique=pbmc_config_translation)

In [17]:
model_translation.train(adatas_train, verbose=True)


training


  0%|          | 0/10 [00:00<?, ?it/s]



Losses                    Value
-------------------  ----------
discriminator_loss   0.0595083
reconstruction_loss  0.0411343
generator_loss       0.00795011
contrastive_loss     9.3067
translation_loss     0.0411343
best_head            0
best model saved at saved_results/thesis/pbmc/translation/CD4T/CD4T/train_best.pt 

model saved at saved_results/thesis/pbmc/translation/CD4T/CD4T/train_epoch_1.pt 



 10%|█         | 1/10 [00:04<00:43,  4.88s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05620258 -0.05620258]
                                [-4.02247524 -4.02247524]]
Confusion Matrix               [[   7    0  626    0    2   61   49]
                                [   0    0  113    0    0  338    2]
                                [  15    0 1863    1   13  245  212]
                                [   3    0  297    1    1   61   27]
                                [   0    0  188    0    0  128   23]
                                [   0    0  565    0    1 1260   33]
                                [   2    0  365    0    1   67   46]]
Accuracy                       0.4801995163240629
Adjusted Rand Index            0.13020679729601353
Normalized Mutual Information  0.1497576428276985


Losses                   Value
-------------------  ---------
discriminator_loss   0.0555364
reconstruction_loss  0.0382109
generator_los

...



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05606649 -0.05606649]
                                [-0.00237421 -0.00237421]]
Confusion Matrix               [[  13    0  274    1    0  457    0]
                                [   0    0  125    0    0  328    0]
                                [  20    1  937    0    0 1391    0]
                                [   2    0   99    0    0  289    0]
                                [   5    0  139    0    0  195    0]
                                [  10    1  637    0    0 1211    0]
                                [   5    0  106    0    0  370    0]]
Accuracy                       0.32663240628778717
Adjusted Rand Index            -0.006246173097579558
Normalized Mutual Information  0.00868536358440616
model saved at saved_results/thesis/pbmc/translation/CD4T/CD4T/train_epoch_9.pt 



 90%|█████████ | 9/10 [00:32<00:03,  3.42s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05606646 -0.05606649]
                                [-0.00198999 -0.00198999]]
Confusion Matrix               [[   4    1  282    0    0  458    0]
                                [   0    0  129    0    0  324    0]
                                [  11    1  956    0    0 1377    4]
                                [   1    0  101    0    0  288    0]
                                [   0    0  150    0    0  189    0]
                                [   7    1  663    0    0 1188    0]
                                [   1    0  108    0    0  371    1]]
Accuracy                       0.3248186215235792
Adjusted Rand Index            -0.006870036655446086
Normalized Mutual Information  0.008766087338492115


Losses                   Value
-------------------  ---------
discriminator_loss   0.0555608
reconstruction_loss  0.0369381
generator

100%|██████████| 10/10 [00:36<00:00,  3.61s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05582297 -0.05582298]
                                [-0.00215461 -0.00215461]]
Confusion Matrix               [[   5    0  275    0    0  463    2]
                                [   2    0  123    0    0  328    0]
                                [  13    0  946    0    0 1384    6]
                                [   1    0  101    0    0  288    0]
                                [   3    0  144    0    1  190    1]
                                [   7    0  635    0    1 1214    2]
                                [   2    0  106    0    0  371    2]]
Accuracy                       0.32769044740024184
Adjusted Rand Index            -0.006267101988215142
Normalized Mutual Information  0.008452413107276518


## Single-task cell type annotation (classification) via UnitedNet

The translation networks' parameters are ignored.

As mentioned above all metrics are monitored. For the classification task we are intested for the:
- Confusion matrix
- Adjusted Rank Index
- Accuracy
- Normalized Mutual information

In [18]:
from unitednet.interface import UnitedNet


pbmc_config_classification = {
    "train_batch_size": 512,
    "finetune_batch_size": 5000,
    "transfer_batch_size": 512,
    "train_epochs": 10,
    "finetune_epochs": 10,
    "transfer_epochs": 20,
    "train_task": "supervised_group_identification_only", # classification
    "finetune_task": None,
    "transfer_task": None,
    "train_loss_weight": None,
    "finetune_loss_weight": None,
    "transfer_loss_weight": None,
    "lr": 0.1,
    "checkpoint": 1,
    "n_head": 1,
    "noise_level":[0,0],
    "fuser_type":"WeightedMean",
    "encoders": [
        {
            "input": features_num,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
        {
            "input": features_num,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
    ],
    "latent_projector": None,
    "decoders": [
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": features_num,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "sigmoid"],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": features_num,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", None],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
    ],
    "discriminators": [
        {
            "input": features_num,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
        {
            "input": features_num,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
    ],
    "projectors": {
        "input": 64,
        "hiddens": [],
        "output": 100,
        "use_biases": [True],
        "dropouts": [0],
        "activations": ["relu"],
        "use_batch_norms": [False],
        "use_layer_norms": [True],
    },
    "clusters": {
        "input": 100,
        "hiddens": [],
        "output": len(cell_type_list),
        "use_biases": [False],
        "dropouts": [0],
        "activations": [None],
        "use_batch_norms": [False],
        "use_layer_norms": [False],
    },
}

device="cuda:0"
test_batch= batch
root_save_path = f"saved_results/thesis/pbmc/classification"

model_classification = UnitedNet(f"{root_save_path}/{test_batch}", device=device, technique=pbmc_config_classification)

In [19]:
model_classification.train(adatas_train, verbose=True)


training


  0%|          | 0/10 [00:00<?, ?it/s]



Losses                        Value
-------------------------  --------
cross_entropy_loss_head_0  0.226991
best_head                  0
best model saved at saved_results/thesis/pbmc/classification/CD4T/CD4T/train_best.pt 

model saved at saved_results/thesis/pbmc/classification/CD4T/CD4T/train_epoch_1.pt 



 10%|█         | 1/10 [00:04<00:37,  4.22s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[ -452.13931274  -348.13018799]
                                [-4292.48925781  -453.43054199]]
Confusion Matrix               [[ 740    0    2    1    2    0    0]
                                [   0  452    0    0    0    1    0]
                                [   6    7 2200  135    0    1    0]
                                [   0    0    0  381    0    0    9]
                                [   0    2    0    0  334    3    0]
                                [   3  860    2    0   64  923    7]
                                [   0    0    1    5    0    0  475]]
Accuracy                       0.8320737605804112
Adjusted Rand Index            0.7605106205207351
Normalized Mutual Information  0.8246357413941272


Losses                         Value
-------------------------  ---------
cross_entropy_loss_head_0  0.0244222
best_head      

...



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[ -447.47564697  -344.02441406]
                                [-3830.68359375  -332.47918701]]
Confusion Matrix               [[ 745    0    0    0    0    0    0]
                                [   0  453    0    0    0    0    0]
                                [   0    0 2349    0    0    0    0]
                                [   0    0    0  390    0    0    0]
                                [   0    0    0    0  339    0    0]
                                [   0   18    0    0    0 1841    0]
                                [   0    0    0    0    0    0  481]]
Accuracy                       0.997279322853688
Adjusted Rand Index            0.9947218098479961
Normalized Mutual Information  0.9919325919933397
model saved at saved_results/thesis/pbmc/classification/CD4T/CD4T/train_epoch_9.pt 



 90%|█████████ | 9/10 [00:38<00:04,  4.35s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[ -449.81253052  -346.45788574]
                                [-4052.74536133  -403.01547241]]
Confusion Matrix               [[ 745    0    0    0    0    0    0]
                                [   0  453    0    0    0    0    0]
                                [   0    0 2349    0    0    0    0]
                                [   0    0    0  390    0    0    0]
                                [   0    0    0    0  339    0    0]
                                [   0   50    0    0    0 1809    0]
                                [   0    0    0    0    0    0  481]]
Accuracy                       0.9924425634824667
Adjusted Rand Index            0.9855006098757555
Normalized Mutual Information  0.9822068752851634


Losses                           Value
-------------------------  -----------
cross_entropy_loss_head_0  0.000172609
best_head

100%|██████████| 10/10 [00:42<00:00,  4.24s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[ -451.26821899  -345.86209106]
                                [-3976.87792969  -398.92633057]]
Confusion Matrix               [[ 745    0    0    0    0    0    0]
                                [   0  453    0    0    0    0    0]
                                [   0    0 2349    0    0    0    0]
                                [   0    0    0  390    0    0    0]
                                [   0    0    0    0  339    0    0]
                                [   0    0    0    0    0 1859    0]
                                [   0    0    0    0    0    0  481]]
Accuracy                       1.0
Adjusted Rand Index            1.0
Normalized Mutual Information  1.0


## Conclusion

Perturbation response prediction is evaluated on a held out cell type group. The model given the control cells of a particular cell type, predicts the stimulated gene expression profiles, and with post-processing analysis, the DEGs are identified. scButterfly, scGen, and scPreGAN, do the comparison based on 1) the common DEGs, and 2) .

The issue in our initial thinking is that the classification task can't be tested with the above setup. To hold out a cell type group to test it for perturbation response, won't be an equivalent meaningful evaluation for the classification, since we will have only one type. At the same time, the cell type knowledge is a prerequisite to train and test the model, so the cell type classification task seems not a meaningful one.

In other words, the UnitedNet's classification task is meaningful, since it is possible from an experiment to have a multiple modality information for a cell, such as knowing the gene, protein or dna accessibility information. But from a perturbation perspective, the knowledge of a paired control and stimulated cell is unknown (we need cell type + optimal transport to create this pseudo information). On the other hand, we could assume that given the control cell, a model can predict the stimulated, and then having both, it could be leveraged for cell type. This would be an interesting use case to try it, and test the classification task, but the limitations of the model in the perturbation response will be carried along as the input for another task. That's we should explore other workflows.

Overall, we could change the way we test, by keeping from all cell types a portion of it for testing, instead of holding out a whole cell type group (as we currently do for the validation set). Having all the above combined, UnitedNet, as it is, with the translation, and classification task can be explored on perturbation modeling.